In [2]:
import os
import sys
import urllib, io

import numpy as np
import scipy.stats as stats
import pandas as pd

import pymongo as pm
from collections import Counter
import json
import re
import ast

from PIL import Image, ImageOps, ImageDraw, ImageFont 

from io import BytesIO
import base64

import  matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

import seaborn as sns
sns.set_context('talk')
sns.set_style('darkgrid')

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [3]:
## directory & file hierarchy
proj_dir = os.path.abspath('..')
datavol_dir = os.path.join(proj_dir,'data')
analysis_dir = os.path.abspath(os.path.join(os.getcwd(),'..'))
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
json_dir = os.path.join(results_dir,'json')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
png_dir = os.path.abspath(os.path.join(datavol_dir,'png'))
jefan_dir = os.path.join(analysis_dir,'jefan')
will_dir = os.path.join(analysis_dir,'will')

## add helpers to python path
if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       

In [4]:
## set vars 
auth = pd.read_csv(os.path.join(analysis_dir,'auth.txt'), header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'cogtoolslab.org' ## cocolab ip address

## have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['block_construction']
coll = db['silhouette']

## which iteration name should we use?
iterationName = 'pilot4'

## Sanity Checks

In [5]:
# Ensure one to one gameID and workerId 
# Should only happen if a repeat worker gets through

query = coll.find({"$and":[
                        {'workerId':{'$exists':True}},
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'trial_end'},
                        {"$or":[{'iterationName':'pilot2'},
                                {'iterationName':'pilot3'},
                                {'iterationName':'pilot4'}]},
                        {'trialNum':0}]
                     })

df_trial_end_full = pd.DataFrame(list(query.sort('timeAbsolute')))
#df_trial_end_full[['workerId','gameID']]


assert (np.mean(df_trial_end_full['workerId'].value_counts()) == np.mean(df_trial_end_full['gameID'].value_counts()))

In [6]:
# get ids of people with trial 15 data
query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'trial_end'},
                        {'iterationName': iterationName},
                        {'trialNum':15}]
                     })
complete_data_df = pd.DataFrame(query)
complete_data_ids = list(complete_data_df['workerId'])

In [7]:
complete_data_df['workerId'].nunique()

35

## Collect data from db and filter with sanity checks

In [41]:
query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'trial_end'},
                        {'iterationName':iterationName}]
                     })

df_trial_end_full = pd.DataFrame(list(query.sort('timeAbsolute')))



# filter dataframe for complete datasets
df_trial_end_full_filtered = df_trial_end_full[df_trial_end_full.workerId.isin(complete_data_ids)]


# reduce to crucial information
df_trial_end_reduced_filtered = df_trial_end_full_filtered[['gameID','trialNum','phase','condition',
                                                            'eventType','score','normedScore','numBlocks',
                                                            'timeAbsolute','timeRelative','buildTime',
                                                            'currBonus','exploreResets','buildResets',
                                                            'allVertices','nPracticeAttempts','exploreStartTime',
                                                            'buildStartTime','buildFinishTime','targetName','numBlocksExplore']]

df = df_trial_end_reduced_filtered.sort_values(by=['gameID', 'timeAbsolute'])


### Integrate reset data before sending

In [42]:
# Resets

query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'reset'},
                        {'phase':'build'},
                        {'iterationName':iterationName}]
                     })
df_resets_full = pd.DataFrame(list(query.sort('timeAbsolute')))
df_resets_full_filtered = df_resets_full[df_resets_full.workerId.isin(complete_data_ids)]
df_resets_reduced_filtered = df_resets_full_filtered[['gameID','trialNum','phase','condition','numBlocks']]
pre_reset_blocks = df_resets_reduced_filtered.groupby(
    ['gameID','trialNum','phase','condition'])['numBlocks'].apply(list).reset_index()

# Merge pre-reset blocks with build data
pre_reset_blocks = pre_reset_blocks.rename(columns = {'numBlocks':'preResetBuildBlocks'})

pre_reset_blocks = pre_reset_blocks.fillna(value={'preResetBuildBlocks': 0})
df = df.merge(pre_reset_blocks, on=['gameID', 'trialNum','phase','condition'], how='left')


# Rename and add totals
df = df.fillna(value={'preResetBuildBlocks': 0})
df = df.rename(columns = {'numBlocks':'finalBuildBlocks'})
df['totalBuildBlocks'] = df['finalBuildBlocks'] + df['preResetBuildBlocks'].apply(np.sum)

In [43]:
df

,gameID,trialNum,phase,condition,eventType,score,normedScore,finalBuildBlocks,timeAbsolute,timeRelative,...,buildResets,allVertices,nPracticeAttempts,exploreStartTime,buildStartTime,buildFinishTime,targetName,numBlocksExplore,preResetBuildBlocks,totalBuildBlocks
0,0468-bb3f0e39-c829-430d-9c75-c538d48abe22,0,build,mental,trial_end,0.03,0.882519,5,1.575564e+12,320950.720,...,2,"[[{'x': 496.5844199549109, 'y': 592.4785727811...",None,1.575563e+12,1.575563e+12,1.575564e+12,hand_selected_015,0,"[0, 1]",6
1,0468-bb3f0e39-c829-430d-9c75-c538d48abe22,1,build,physical,trial_end,0.03,0.401916,2,1.575564e+12,416116.955,...,1,"[[{'x': 501.0962572421163, 'y': 592.4968160271...",None,1.575564e+12,1.575564e+12,1.575564e+12,hand_selected_006,0,[5],7
2,0468-bb3f0e39-c829-430d-9c75-c538d48abe22,2,build,physical,trial_end,0.06,0.879605,7,1.575564e+12,511102.920,...,0,"[[{'x': 273.84901599820495, 'y': 592.692045004...",None,1.575564e+12,1.575564e+12,1.575564e+12,hand_selected_004,2,0,7
3,0468-bb3f0e39-c829-430d-9c75-c538d48abe22,3,build,mental,trial_end,0.06,0.557653,10,1.575564e+12,606017.875,...,0,"[[{'x': 414.4062149949713, 'y': 482.4785754458...",None,1.575564e+12,1.575564e+12,1.575564e+12,hand_selected_001,0,0,10
4,0468-bb3f0e39-c829-430d-9c75-c538d48abe22,4,build,mental,trial_end,0.11,0.976224,7,1.575564e+12,701017.770,...,0,"[[{'x': 444.9290823724809, 'y': 592.8611021306...",None,1.575564e+12,1.575564e+12,1.575564e+12,hand_selected_005,0,0,7
5,0468-bb3f0e39-c829-430d-9c75-c538d48abe22,5,build,physical,trial_end,0.16,0.972015,8,1.575564e+12,795937.875,...,0,"[[{'x': 495.3612855521121, 'y': 592.6425815132...",None,1.575564e+12,1.575564e+12,1.575564e+12,hand_selected_013,1,0,8
6,0468-bb3f0e39-c829-430d-9c75-c538d48abe22,6,build,physical,trial_end,0.16,0.407773,5,1.575564e+12,890831.065,...,1,"[[{'x': 299.40381861754133, 'y': 647.532996891...",None,1.575564e+12,1.575564e+12,1.575564e+12,hand_selected_016,3,[1],6
7,0468-bb3f0e39-c829-430d-9c75-c538d48abe22,7,build,mental,trial_end,0.19,0.920874,8,1.575564e+12,986217.220,...,0,"[[{'x': 273.7809319900754, 'y': 592.6615456318...",None,1.575564e+12,1.575564e+12,1.575564e+12,hand_selected_007,0,0,8
8,0468-bb3f0e39-c829-430d-9c75-c538d48abe22,8,build,physical,trial_end,0.19,0.582343,7,1.575564e+12,1081559.195,...,0,"[[{'x': 309.8475213415538, 'y': 592.4968163005...",None,1.575564e+12,1.575564e+12,1.575564e+12,hand_selected_014,4,0,7
9,0468-bb3f0e39-c829-430d-9c75-c538d48abe22,9,build,mental,trial_end,0.22,0.885490,8,1.575564e+12,1176330.575,...,0,"[[{'x': 273.848922537562, 'y': 592.72644182711...",None,1.575564e+12,1.575564e+12,1.575564e+12,hand_selected_009,0,0,8


In [9]:
## save out to csv dir, where all the csv's go to live
out_path = os.path.join(csv_dir,'block_silhouette_{}.csv'.format(iterationName))
df_for_analysis.to_csv(out_path)

## Settled Block Data

In [16]:
print('Loading iteration: ' + iterationName)
query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'settled'},
                        {'iterationName':iterationName}]
                     })

df_settled_full = pd.DataFrame(list(query))


# filter dataframe for complete datasets
df_settled_full_filtered = df_settled_full[df_settled_full.workerId.isin(complete_data_ids)]

print('Loaded ' + str(df_settled_full_filtered.shape[0]) + ' complete sets of settled blocks')
# reduce to crucial information

Loading iteration: pilot4
Loaded 4851 complete sets of settled blocks


In [17]:

df_settled_reduced_filtered = df_settled_full_filtered[['gameID','trialNum','phase','condition',
                                                            'eventType','numBlocks', 'timeAbsolute','timeRelative',
                                                            'normedScore','currBonus','score','incrementalScore','normedIncrementalScore',
                                                            'currBonus','allVertices','targetName','relativePlacementTime','iterationName',
                                                            'blockKind'
                                                           ]]

df_settled_reduced_filtered = df_settled_reduced_filtered.sort_values(by=['gameID', 'timeAbsolute'])

buildstart = df_for_analysis[['gameID','trialNum','buildStartTime','exploreStartTime']]

# copy across time variables that were not saved in the correct place in pilot 3
df_settled_reduced_filtered = df_settled_reduced_filtered.merge(buildstart, on=['gameID', 'trialNum'], how='left')
df_settled_reduced_filtered['timePlaced'] = df_settled_reduced_filtered['timeAbsolute'] - df_settled_reduced_filtered['buildStartTime']
df_settled_reduced_filtered.loc[(df_settled_reduced_filtered.timePlaced < 0),'timePlaced'] = df_settled_reduced_filtered[df_settled_reduced_filtered.timePlaced < 0]['timeAbsolute'] - df_settled_reduced_filtered[df_settled_reduced_filtered.timePlaced < 0]['exploreStartTime']

# 
df_settled_reduced_filtered.loc[(df_settled_reduced_filtered.iterationName == 'pilot4'),'timePlaced'] = df_settled_reduced_filtered[(df_settled_reduced_filtered.iterationName == 'pilot4')]['relativePlacementTime'] 
df_settled_reduced_filtered.loc[(df_settled_reduced_filtered.timePlaced <= 0),'timePlaced'] = 0



In [18]:
## save out to csv dir, where all the csv's go to live
out_path = os.path.join(csv_dir,'block_silhouette_settled_{}.csv'.format(iterationName))
df_settled_reduced_filtered.to_csv(out_path)

In [19]:
## which iteration name should we use?
iterationName = 'pilot4'

## load in dataframe
data_path = os.path.join(csv_dir,'block_silhouette_settled_{}.csv'.format(iterationName))
df = pd.read_csv(data_path)
df

,Unnamed: 0,gameID,trialNum,phase,condition,eventType,numBlocks,timeAbsolute,timeRelative,normedScore,...,normedIncrementalScore,currBonus.1,allVertices,targetName,relativePlacementTime,iterationName,blockKind,buildStartTime,exploreStartTime,timePlaced
0,0,0468-bb3f0e39-c829-430d-9c75-c538d48abe22,0,build,mental,settled,1,1.575564e+12,304021.925,0.000000,...,0.406914,0.00,"[[{'x': 496.5999450683595, 'y': 592.4785714285...",hand_selected_015,38477,pilot4,E,1.575563e+12,1.575563e+12,38477
1,1,0468-bb3f0e39-c829-430d-9c75-c538d48abe22,0,build,mental,settled,2,1.575564e+12,309074.700,0.000000,...,0.589698,0.00,"[[{'x': 496.5999450683595, 'y': 592.4785714285...",hand_selected_015,43294,pilot4,C,1.575563e+12,1.575563e+12,43294
2,2,0468-bb3f0e39-c829-430d-9c75-c538d48abe22,0,build,mental,settled,3,1.575564e+12,314450.495,0.000000,...,0.740677,0.00,"[[{'x': 496.60188322094706, 'y': 592.494480207...",hand_selected_015,48380,pilot4,E,1.575563e+12,1.575563e+12,48380
3,3,0468-bb3f0e39-c829-430d-9c75-c538d48abe22,0,build,mental,settled,4,1.575564e+12,317490.745,0.000000,...,0.884437,0.00,"[[{'x': 496.5941205868618, 'y': 592.5468387980...",hand_selected_015,53738,pilot4,E,1.575563e+12,1.575563e+12,53738
4,4,0468-bb3f0e39-c829-430d-9c75-c538d48abe22,0,build,mental,settled,5,1.575564e+12,320888.670,0.882519,...,0.882519,0.03,"[[{'x': 496.5844199549109, 'y': 592.4785727811...",hand_selected_015,56779,pilot4,E,1.575563e+12,1.575563e+12,56779
5,5,0468-bb3f0e39-c829-430d-9c75-c538d48abe22,1,build,physical,settled,1,1.575564e+12,372582.030,0.000000,...,0.285125,0.00,"[[{'x': 498.8499450683594, 'y': 592.4785714285...",hand_selected_006,10837,pilot4,E,1.575564e+12,1.575564e+12,10837
6,6,0468-bb3f0e39-c829-430d-9c75-c538d48abe22,1,build,physical,settled,2,1.575564e+12,381065.565,0.000000,...,0.477591,0.00,"[[{'x': 498.8499450683594, 'y': 592.4785714285...",hand_selected_006,16695,pilot4,B,1.575564e+12,1.575564e+12,16695
7,7,0468-bb3f0e39-c829-430d-9c75-c538d48abe22,1,build,physical,settled,3,1.575564e+12,386365.910,0.000000,...,0.633159,0.00,"[[{'x': 498.8499450683594, 'y': 592.4785714285...",hand_selected_006,25184,pilot4,E,1.575564e+12,1.575564e+12,25184
8,8,0468-bb3f0e39-c829-430d-9c75-c538d48abe22,1,build,physical,settled,4,1.575564e+12,392854.330,0.000000,...,0.626784,0.00,"[[{'x': 498.8499450683594, 'y': 592.4785714285...",hand_selected_006,30484,pilot4,D,1.575564e+12,1.575564e+12,30484
9,9,0468-bb3f0e39-c829-430d-9c75-c538d48abe22,1,build,physical,settled,1,1.575564e+12,410077.425,0.000000,...,0.389452,0.00,"[[{'x': 501.0999450683595, 'y': 592.4785714285...",hand_selected_006,50687,pilot4,E,1.575564e+12,1.575564e+12,50687


## Explore End

In [132]:
# Ensure one to one gameID and workerId 
# Should only happen if a repeat worker gets through

query = coll.find({"$and":[
                        {'workerId':{'$exists':True}},
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'explore_end'},
                        {"$or":[{'iterationName':'pilot2'},
                                {'iterationName':'pilot3'},
                                {'iterationName':'pilot4'}]},
                        {'trialNum':0}]
                     })

df_explore_end_full = pd.DataFrame(list(query.sort('timeAbsolute')))

assert (np.mean(df_trial_end_full['workerId'].value_counts()) == np.mean(df_trial_end_full['gameID'].value_counts()))

In [133]:
# get ids of people with trial 15 data
query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'explore_end'},
                        {'iterationName': iterationName},
                        {'trialNum':15}]
                     })
complete_data_df = pd.DataFrame(query)
complete_data_ids = list(complete_data_df['workerId'])

In [134]:
query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'explore_end'},
                        {'iterationName':iterationName}]
                     })

df_explore_end_full = pd.DataFrame(list(query.sort('timeAbsolute')))



# filter dataframe for complete datasets
df_explore_end_full_filtered = df_explore_end_full[df_explore_end_full.workerId.isin(complete_data_ids)]


# reduce to crucial information
df_explore_end_reduced_filtered = df_explore_end_full_filtered[['gameID','trialNum','phase','condition',
                                                            'eventType','score','normedScore','numBlocks',
                                                            'timeAbsolute','timeRelative',
                                                            'currBonus','exploreResets',
                                                            'allVertices','nPracticeAttempts','exploreStartTime',
                                                            'targetName','numBlocksExplore']]

df_explore = df_explore_end_reduced_filtered.sort_values(by=['gameID', 'timeAbsolute'])

In [135]:
df_explore

,gameID,trialNum,phase,condition,eventType,score,normedScore,numBlocks,timeAbsolute,timeRelative,currBonus,exploreResets,allVertices,nPracticeAttempts,exploreStartTime,targetName,numBlocksExplore
499,0468-bb3f0e39-c829-430d-9c75-c538d48abe22,0,explore,mental,explore_end,0.00,0,0,1.575563e+12,260788.145,0,0,[],None,1.575563e+12,hand_selected_015,0
507,0468-bb3f0e39-c829-430d-9c75-c538d48abe22,1,explore,physical,explore_end,0.03,0,0,1.575564e+12,355975.730,0,0,[],None,1.575564e+12,hand_selected_006,0
513,0468-bb3f0e39-c829-430d-9c75-c538d48abe22,2,explore,physical,explore_end,0.03,0,2,1.575564e+12,450947.310,0,0,"[[{'x': 273.85123209670365, 'y': 592.491506380...",None,1.575564e+12,hand_selected_004,2
519,0468-bb3f0e39-c829-430d-9c75-c538d48abe22,3,explore,mental,explore_end,0.06,0,0,1.575564e+12,545863.365,0,0,[],None,1.575564e+12,hand_selected_001,0
524,0468-bb3f0e39-c829-430d-9c75-c538d48abe22,4,explore,mental,explore_end,0.06,0,0,1.575564e+12,640864.505,0,0,[],None,1.575564e+12,hand_selected_005,0
529,0468-bb3f0e39-c829-430d-9c75-c538d48abe22,5,explore,physical,explore_end,0.11,0,1,1.575564e+12,735809.805,0,1,"[[{'x': 499.8499450683595, 'y': 592.4785714285...",None,1.575564e+12,hand_selected_013,1
533,0468-bb3f0e39-c829-430d-9c75-c538d48abe22,6,explore,physical,explore_end,0.16,0,3,1.575564e+12,830666.220,0,1,"[[{'x': 553.8554722644245, 'y': 592.4919440962...",None,1.575564e+12,hand_selected_016,3
537,0468-bb3f0e39-c829-430d-9c75-c538d48abe22,7,explore,mental,explore_end,0.16,0,0,1.575564e+12,926050.090,0,0,[],None,1.575564e+12,hand_selected_007,0
542,0468-bb3f0e39-c829-430d-9c75-c538d48abe22,8,explore,physical,explore_end,0.19,0,4,1.575564e+12,1021410.030,0,0,"[[{'x': 177.09994506835938, 'y': 592.478571428...",None,1.575564e+12,hand_selected_014,4
547,0468-bb3f0e39-c829-430d-9c75-c538d48abe22,9,explore,mental,explore_end,0.19,0,0,1.575564e+12,1116169.525,0,0,[],None,1.575564e+12,hand_selected_009,0


## Integrate reset data before sending

In [136]:
# Resets

query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'reset'},
                        {'phase':'explore'},
                        {'iterationName':iterationName}]
                     })
df_resets_full = pd.DataFrame(list(query.sort('timeAbsolute')))
df_resets_full_filtered = df_resets_full[df_resets_full.workerId.isin(complete_data_ids)]
df_resets_reduced_filtered = df_resets_full_filtered[['gameID','trialNum','phase','condition','numBlocks']]
pre_reset_blocks = df_resets_reduced_filtered.groupby(
    ['gameID','trialNum','phase','condition'])['numBlocks'].apply(list).reset_index()

# Merge pre-reset blocks with explore data
pre_reset_blocks = pre_reset_blocks.rename(columns = {'numBlocks':'preResetExploreBlocks'})
df_explore = df_explore.merge(pre_reset_blocks, on=['gameID', 'trialNum','phase','condition'], how='left')

# Rename and add totals
df_explore.loc[df_explore.preResetExploreBlocks.isnull(),'preResetExploreBlocks'] = df_explore.preResetExploreBlocks.loc[df_explore.preResetExploreBlocks.isnull()].apply(lambda x:[])
df_explore = df_explore.rename(columns = {'numBlocksExplore':'finalExploreBlocks'})
df_explore['totalExploreBlocks'] = df_explore['finalExploreBlocks'] + df_explore['preResetExploreBlocks'].apply(np.sum)

#df_explore = df_explore.fillna(value={'totalExploreBlocks': 0 })


In [142]:
df_explore['numAttempts'] = df_explore['preResetExploreBlocks'].apply(len) + 1

In [143]:
## save out to csv dir, where all the csv's go to live
out_path = os.path.join(csv_dir,'block_silhouette_explore_{}.csv'.format(iterationName))
df_explore.to_csv(out_path)

### update numBlocksExplore in trial_end